In [2]:
#Load Libraries
from oauth2client.service_account import ServiceAccountCredentials
from googleapiclient.discovery import build
import httplib2
import pandas as pd

In [47]:
SCOPES = ['https://www.googleapis.com/auth/analytics.readonly']
KEY_FILE_LOCATION = 'auth/service_account.json'
VIEW_ID = '246084583'

In [48]:
start_date = '2021-07-01'
end_date = '2022-06-30'

credentials = ServiceAccountCredentials.from_json_keyfile_name(KEY_FILE_LOCATION, SCOPES)

# Build the service object.
analytics = build('analyticsreporting', 'v4', credentials=credentials)

ga_data = analytics.reports().batchGet(body={
    "reportRequests": [{
        "viewId": VIEW_ID,
        "pageSize": 1000000,
        "dateRanges": [{"startDate": start_date, "endDate": end_date}],
        "metrics": [
            {"expression": "ga:users"},
            {"expression": "ga:sessions"},
            {"expression": "ga:goal4Completions"},
            {"expression": "ga:goal5Completions"}
        ], 
        "dimensions": [
            {"name": "ga:date"},
            {"name": "ga:campaign"},
            {"name": "ga:source"},
            {'name': 'ga:medium'}
        ]
        # "dimensionFilterClauses": [
        #   {"filters": [{
        #     "dimensionName": "ga:medium",
        #     "operator": "REGEXP",
        #     "expressions": ["cpc"]
        #     }]}, 
        #   {"filters": [{
        #     "dimensionName": "ga:source",
        #     "operator": "REGEXP",
        #     "expressions": ["google|bing"]
        #     }]}
        # ]
    }]}).execute()

In [49]:
# Funktion für die Umwandlung der Response (list) in ein Dataframe

def ga_response_dataframe(response):
    row_list = []
    # Get each collected report
    for report in response.get('reports', []):
        # Set column headers
        column_header = report.get('columnHeader', {})
        dimension_headers = column_header.get('dimensions', [])
        metric_headers = column_header.get('metricHeader', {}).get('metricHeaderEntries', [])
    
        # Get each row in the report
        for row in report.get('data', {}).get('rows', []):
            # create dict for each row
            row_dict = {}
            dimensions = row.get('dimensions', [])
            date_range_values = row.get('metrics', [])

            # Fill dict with dimension header (key) and dimension value (value)
            for header, dimension in zip(dimension_headers, dimensions):
                row_dict[header] = dimension

            # Fill dict with metric header (key) and metric value (value)
            for i, values in enumerate(date_range_values):
                for metric, value in zip(metric_headers, values.get('values')):
                # Set int as int, float a float
                    if ',' in value or '.' in value:
                        row_dict[metric.get('name')] = float(value)
                    else:
                        row_dict[metric.get('name')] = int(value)

            row_list.append(row_dict)
    # return (row_list)
    df = pd.DataFrame(row_list)
    df.columns = df.columns.str.replace('ga:', '', regex=True)
    return df

In [37]:
del (start_date, end_date, VIEW_ID, KEY_FILE_LOCATION, SCOPES, credentials, analytics)

In [50]:
ga_data = ga_response_dataframe(ga_data)

In [ ]:
ga_data.head()

In [53]:
ga_data.rename({'goal4Completions':'jetzt_bestellen', 'goal5Completions':'kennenlernset_anfordern'}, axis='columns', inplace=True)